# THE MPS CALCULATOR

In [5]:
using Pkg
Pkg.add("ITensors")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [1]:
using LinearAlgebra
using Plots: plot, savefig




function mps_function(psi::Vector{ComplexF64}, cutoff::Float64)
    n = round(Int, log2(length(psi)))  # Number of qubits
    @assert 2^n == length(psi) "Length of psi must be a power of 2"

    mps = []
    D_prev = 1 # Initial bond dimension

    for k in 1:n-1
        d_left = D_prev
        d_phys = 2 # Physical dimension (2 for qubits)
        d_right = length(psi) ÷ (d_left * d_phys)

        psi_matrix = reshape(psi, (d_left * d_phys, d_right))
        U, S, V = svd(psi_matrix)
        
       
      
        if k == ceil(Int, n/2)
            global middle_singular_values = copy(S)
            plt = plot(
                S,
                marker = :circle,
                xlabel = "Number of Singular Values",
                ylabel = "Singular Values",
                title = "Singular Values at Step $k (Middle Site)",
                yscale = :log10,
                ylims = (1e-25, 1e5),
                #yformatter = y -> @sprintf("%.5e", y),   # <--- show 2 decimal digits in scientific notation
                legend = false
            )
            savefig(plt, "singular_values_plot(FGS).pdf")
        end

        

        # Find how many singular values are above cutoff
        r = count(s -> abs(s) > cutoff, S)
        if r == 0
            error("All singular values below cutoff at step $k — increase cutoff or check state")
        end

        # Truncate U, S, V to rank r  (non zero singular values)
        U_trunc = U[:, 1:r]
        S_trunc = S[1:r]
        V_trunc = V[:, 1:r]

        A = reshape(U_trunc, d_left, d_phys, r)
        push!(mps, A)

        psi = Diagonal(S_trunc) * V_trunc'
        D_prev = r
    end

    # Final tensor
    A_last = reshape(psi, D_prev, 2, 1)
    push!(mps, A_last)

    return mps
end



mps_function (generic function with 1 method)

# READING COMPLEX TEXT FILE (2 Functions) 

In [3]:
function load_complex_vector_from_txt(filename::String)
    psi = ComplexF64[]
    open(filename, "r") do file
        for line in eachline(file)
            s = replace(line, "im" => "")  # remove "im"
            parts = split(s, "+")
            if length(parts) == 2
                real_part = parse(Float64, strip(parts[1]))
                imag_part = parse(Float64, strip(parts[2]))
                push!(psi, ComplexF64(real_part, imag_part))
            end
        end
    end
    return psi
end

load_complex_vector_from_txt (generic function with 1 method)

In [4]:
function load_complex_vector_from_txt_2(filename::String)
    psi = ComplexF64[]
    open(filename, "r") do file
        for line in eachline(file)
            # Remove "im" and index number
            parts = split(strip(line), r"\s+")
            if length(parts) >= 4
                real_str = parts[1+1]  # after the index
                sign = parts[2+1]      # '+' or '-'
                imag_str = parts[3+1]

                real_part = parse(Float64, real_str)
                imag_part = parse(Float64, imag_str)
                imag_part *= (sign == "+" ? 1 : -1)

                push!(psi, ComplexF64(real_part, imag_part))
            end
        end
    end
    return psi
end

load_complex_vector_from_txt_2 (generic function with 1 method)

# APLLYING THE MPS CALCULATER ON DIFFERENT STATEVECTORS

In [20]:
psi = load_complex_vector_from_txt("cl.txt")
MPS = mps_function(psi, 1e-8)
for (i, A) in enumerate(MPS)
    println("A[$i] shape: ", size(A))
end

┌ Warning: Invalid negative or zero value 0.0 found at series index 5 for log10 based yscale
└ @ Plots /Users/arya/.julia/packages/Plots/MR7sb/src/utils.jl:106


A[1] shape: (1, 2, 2)
A[2] shape: (2, 2, 4)
A[3] shape: (4, 2, 4)
A[4] shape: (4, 2, 4)
A[5] shape: (4, 2, 4)
A[6] shape: (4, 2, 4)
A[7] shape: (4, 2, 4)
A[8] shape: (4, 2, 2)
A[9] shape: (2, 2, 1)
A[10] shape: (1, 2, 1)


## Renyi Entropy

In [9]:
function renyi_entropies(S::AbstractVector)
    p = S.^2
    p ./= sum(p)
    Ren_En = zeros(Float64, 10)
    #Van Neumann Entropy
    Ren_En[1] = -sum(pi == 0 ? 0.0 : pi * log(pi) for pi in p)          #Note to myself: Ternary Operator
    #Renyi Entropies
    for n in 2:10
        Ren_En[n] = (1 / (1 - n)) * log(sum(p .^ n))
    end

    return Ren_En
end

renyi_entropies (generic function with 1 method)

#### Checking the singular values

In [21]:
middle_singular_values

8-element Vector{Float64}:
 0.5000008618312572
 0.5000008618312572
 0.5000008618312572
 0.5000008618312572
 0.0
 0.0
 0.0
 0.0

#### Renyi Entropy of Haar random (2-10)

In [10]:
E1 = renyi_entropies(middle_singular_values)

10-element Vector{Float64}:
 2.961076670217562
 2.770223215897558
 2.659049497666836
 2.5853417650600177
 2.5331649774307654
 2.4945419376039455
 2.4649184727989355
 2.4415142316165874
 2.422555093265853
 2.4068680729610508

#### Renyi Entropy of Stablizer (2-10)

In [22]:
#Clifford_10_4
E2 = renyi_entropies(middle_singular_values)


10-element Vector{Float64}:
 1.3862943611198906
 1.3862943611198906
 1.3862943611198906
 1.3862943611198904
 1.3862943611198906
 1.3862943611198908
 1.3862943611198904
 1.3862943611198906
 1.3862943611198906
 1.3862943611198906

# SERs

### Building the Pauli Strings

In [ ]:
#For Cunstructing the  Pauli charactristic distribution of MPS, we need the MPO version of the Pauli strings.
using ITensors
using Random
using StatsBase

# Mapping Itensors Operator name to characters
function operator_name(c::Char)
    c == "I" && return "Id"    # Note to myself: && --> short-circuit evaluation
    c == "X" && return "Sx"
    c == "Y" && return "Sy"
    c == "Z" && return "Sz"
    error("Unknown operator character: $c")
end

# Generating a Ramdom Pauli String
function random_pauli_string(n::Int; avoid_all_I::Bool=true, rng=Random.default_rng())
    pauli_ops = ['I', 'X', 'Y', 'Z']
    while true
        s = String(rand(rng, pauli_ops, n))
        if !avoid_all_I || s != repeat("I", n)
            return s
        end
    end
end 

# Generating the MPO for a given Pauli String



Base.Meta.ParseError: ParseError:
# Error @ /Users/arya/Documents/Python/PRJ/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X40sZmlsZQ==.jl:5:10
# Mapping Itensors Operator name to characters
function operator
#        └──────┘ ── Invalid signature in function definition

# RECONSTRUCTING  THE STATEVECTOR BY CONTRACTING MPS ELEMENTS

In [2]:
using TensorOperations

function contract_mps(MPS::Vector)
    psi = MPS[1]  # Start with the first site tensor
0
    for i in 2:length(MPS)
        T = MPS[i]

        
        @tensor psi_new[a, p1, p2, c] := psi[a, p1, b] * T[b, p2, c]

        
        d1 = size(psi, 2)
        d2 = size(T, 2)
        right_bond = size(psi_new, 4)

        psi = reshape(psi_new, 1, d1 * d2, right_bond)
    end

    
    return reshape(psi, :)
end

contract_mps (generic function with 1 method)

# CUTTING OFF THE COMPLEX NUMBERS 

In [1]:
function cutoff_complex_parts(A::Array, cutoff::Float64 = 1e-12)
    re = real.(A)
    im = imag.(A)

    re[abs.(re) .< cutoff] .= 0.0
    im[abs.(im) .< cutoff] .= 0.0

    A .= complex.(re, im)
end

cutoff_complex_parts (generic function with 2 methods)

# APPLYING THE RECONSTRUCTING FUNCTION

In [35]:
re_psi_1 = contract_mps(MPS)
re_psi = cutoff_complex_parts(re_psi_1, 1e-8)



4096-element Vector{ComplexF64}:
    0.02221278104648671 + 0.0im
                    0.0 + 0.0im
                    0.0 + 0.0im
   0.006459266754389635 - 0.013653612995951827im
                    0.0 + 0.0im
   -0.01564549559844753 - 0.010153208000613503im
   0.014734130944164855 + 0.011194121942464162im
                    0.0 + 0.0im
                    0.0 + 0.0im
 -0.0003580795991152047 + 0.009868844424071804im
                        ⋮
                    0.0 + 0.0im
                    0.0 + 0.0im
   0.005425202135683498 - 0.006361943170859172im
   0.002326104433596305 - 0.0015250884178343662im
                    0.0 + 0.0im
  -0.011019829568911775 + 0.0109105172658617im
                    0.0 + 0.0im
                    0.0 + 0.0im
    0.00718388943285008 + 0.0im

# CHECKING THE RECONSTRUCTED STATEVECTOR IS SAME AS INITIAL ONE

In [36]:
norm(re_psi - psi)

1.6981183086019288e-7